<h1 style="font-size:200%; font-family:cursive; color:navy;">1. Import Required Libraries & Dataset</h1>

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
!pip install transformers
import transformers
from transformers import AutoModel, BertTokenizerFast

device = torch.device("cuda")

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
Mounted at /content/drive


<ul>
    <li style="font-size:150%;">The dataset consists of two columns – “label” and “text”. The column “text” contains the message body and the “label” is a binary variable where 1 means spam and 0 means the message is not a spam.</li>
</ul>

In [ ]:
import pandas as pd
import os
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import glob
import random
from typing import List, Tuple
import numpy as np
import pandas as pd
import re # regular expressions
from sklearn.model_selection import train_test_split

import tensorflow as tf

import json

In [ ]:
Segment = str
Genome = List[Segment]

drive_path = "/content/drive/MyDrive/Virus-DNA-classification-BERT-main/old_code/"

metadata_file = glob.glob(drive_path + "data10/*.tsv")
fasta_file = glob.glob(drive_path + "data10/*.fasta")



In [ ]:
total_covid_df = pd.DataFrame(columns = ['accession_id', 'lineage', 'who'])

# Parse the metadata tsv into a data frame
for md_file in metadata_file:
    genome_df = pd.read_csv(md_file, sep='\t')
    genome_df = genome_df.filter(items=['accession_id', 'lineage', 'who'])
    genome_df['lineage'].replace('', np.nan, inplace=True)
    genome_df['lineage'].replace('lineage', np.nan, inplace=True)
    total_covid_df = pd.concat([total_covid_df, genome_df], ignore_index=True)

total_covid_df.dropna(subset=['lineage'], inplace=True)
total_covid_df.reset_index(drop=True, inplace=True)

genomes = [None] * total_covid_df.shape[0]

print(total_covid_df.who.unique())

['Delta' 'Omicron' 'Alpha' 'Epsilon' 'Iota' 'Gamma' 'Beta' 'Mu' 'Eta'
 'Lambda']


In [ ]:
label_mapping = {'Delta':0,'Omicron':1,'Alpha':2,'Epsilon':3,'Iota':4,'Gamma':5, 'Beta':6, 'Mu':7, 'Eta':8, 'Lambda':9}

numeric_labels = [label_mapping[label] for label in total_covid_df.who]
total_covid_df['label'] = numeric_labels
print(total_covid_df)

     accession_id    lineage     who  label
0        LC643052      AY.29   Delta      0
1        MZ069676  B.1.617.2   Delta      0
2        MZ179942  B.1.617.2   Delta      0
3        MZ084552  B.1.617.2   Delta      0
4        LC643047      AY.29   Delta      0
...           ...        ...     ...    ...
4995     MZ936787       C.37  Lambda      9
4996     MZ090034       C.37  Lambda      9
4997     MZ154746       C.37  Lambda      9
4998     MZ278968       C.37  Lambda      9
4999     MZ999204       C.37  Lambda      9

[5000 rows x 4 columns]


In [ ]:
# Access genome data and save in relevant row by accession_id
for f_file in fasta_file:
    fasta_read = open(f_file,"r")
    pattern = re.compile("\|([^|]+)\|")

    current_accession_id = ''
    genome_index = ''
    combined_string = ''

    for i, line in enumerate(fasta_read):
        result = re.search(pattern, line)
        if result is not None:
            if i != 0:
                genomes[genome_index] = combined_string
            current_accession_id = result.group(1)
            genome_index = total_covid_df.index[total_covid_df['accession_id'] == current_accession_id].tolist()[0]
            combined_string = ''
        else:
            combined_string += line.strip()
    genomes[genome_index] = combined_string
    fasta_read.close()

total_covid_df['genome'] = genomes

# printing data
total_covid_df.tail(5)

,accession_id,lineage,who,label,genome
4995,MZ936787,C.37,Lambda,9,GCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAG...
4996,MZ090034,C.37,Lambda,9,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
4997,MZ154746,C.37,Lambda,9,AACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAAT...
4998,MZ278968,C.37,Lambda,9,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...
4999,MZ999204,C.37,Lambda,9,AACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAAT...


In [ ]:
total_covid_df = total_covid_df.drop(columns='who')
total_covid_df = total_covid_df.drop(columns='accession_id')
total_covid_df = total_covid_df.drop(columns='lineage')

In [ ]:
total_covid_df.tail()

,label,genome
4995,9,GCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAG...
4996,9,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
4997,9,AACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAAT...
4998,9,ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACT...
4999,9,AACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAAT...


In [ ]:
# creating kmers of size 3 from the input
KMER = 6

def kmer_fun(seq, K):
    kmer_list = []
    for x in range(len(seq) - K + 1):  # move a window of size K across the sequence
        kmer_list.append(seq[x : x + K])

    kmer_seq = " ".join(kmer_list)
    return kmer_seq

kmer_seqs = total_covid_df['genome'].apply(lambda x: kmer_fun(x, KMER))
kmer_seqs[:5]

0    AAAGGT AAGGTT AGGTTT GGTTTA GTTTAT TTTATA TTAT...
1    AGATCT GATCTG ATCTGT TCTGTT CTGTTC TGTTCT GTTC...
2    AGATCT GATCTG ATCTGT TCTGTT CTGTTC TGTTCT GTTC...
3    TAAAGG AAAGGT AAGGTT AGGTTT GGTTTA GTTTAT TTTA...
4    ATACCT TACCTT ACCTTC CCTTCC CTTCCC TTCCCA TCCC...
Name: genome, dtype: object

In [ ]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(kmer_seqs, total_covid_df['label'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=total_covid_df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

In [ ]:
drive_path = "/content/drive/MyDrive/Virus-DNA-classification-BERT-main/old_code/"
with open(drive_path + 'train_text.pkl', 'wb') as f:
    pickle.dump(train_text, f)
with open(drive_path + 'temp_text.pkl', 'wb') as f:
    pickle.dump(temp_text, f)
with open(drive_path + 'train_labels.pkl', 'wb') as f:
    pickle.dump(train_labels, f)
with open(drive_path + 'temp_labels.pkl', 'wb') as f:
    pickle.dump(temp_labels, f)

with open(drive_path + 'val_text.pkl', 'wb') as f:
    pickle.dump(val_text, f)
with open(drive_path + 'test_text.pkl', 'wb') as f:
    pickle.dump(test_text, f)
with open(drive_path + 'val_labels.pkl', 'wb') as f:
    pickle.dump(val_labels, f)
with open(drive_path + 'test_labels.pkl', 'wb') as f:
    pickle.dump(test_labels, f)

<h1 style="font-size:200%; font-family:cursive; color:navy;">4. Tokenize & Encode the Sequences</h1>

<u><h2 style="font-size:170%; font-family:cursive;">Which Tokenization strategy is used by BERT?</h2></u>

<p style="font-size:150%; font-family:verdana;">BERT uses WordPiece tokenization. The vocabulary is initialized with all the individual characters in the language, and then the most frequent/likely combinations of the existing words in the vocabulary are iteratively added.</p>
<br>
<u><h2 style="font-size:170%; font-family:cursive;">What is the maximum sequence length of the input?</h2></u>

<p style="font-size:150%; font-family:verdana;">The maximum sequence length of the input = 512</p>

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.explode().tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


<h1 style="font-size:200%; font-family:cursive; color:navy;">5. List to Tensors</h1>

In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

<h1 style="font-size:200%; font-family:cursive; color:navy;">6. Data Loader</h1>

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

<h1 style="font-size:200%; font-family:cursive; color:navy;">7. Model Architecture</h1>

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,10)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),lr = 1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
print(np.unique(train_labels))
#compute the class weights
class_weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_labels), y = train_labels)

print("Class Weights:",class_weights)

[0 1 2 3 4 5 6 7 8 9]
Class Weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:

# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights)

# number of training epochs
epochs = 10

<h1 style="font-size:200%; font-family:cursive; color:navy;">8. Fine - Tune</h1>

In [ ]:
# function to train the model
def train():

    model.train()
    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients
        model.zero_grad()

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

      # predictions are in the form of (no. of batches, size of batch, no. of classes).
      # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():

    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:

            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    #evaluate model
    valid_loss, _ = evaluate()

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.305
Validation Loss: 2.304

 Epoch 2 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.306
Validation Loss: 2.304

 Epoch 3 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.304
Validation Loss: 2.303

 Epoch 4 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.303
Validation Loss: 2.303

 Epoch 5 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.305
Validation Loss: 2.303

 Epoch 6 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.305
Validation Loss: 2.303

 Epoch 7 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.306
Validation Loss: 2.303

 Epoch 8 / 100
  Batch    50  of    110.
  Batch   100  of    110.

Evaluating...

Training Loss: 2.304

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

<h1 style="font-size:200%; font-family:cursive; color:navy;">9. Make Predictions</h1>

In [ ]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.10      1.00      0.18        75
           1       0.00      0.00      0.00        75
           2       0.00      0.00      0.00        75
           3       0.00      0.00      0.00        75
           4       0.00      0.00      0.00        75
           5       0.00      0.00      0.00        75
           6       0.00      0.00      0.00        75
           7       0.00      0.00      0.00        75
           8       0.00      0.00      0.00        75
           9       0.00      0.00      0.00        75

    accuracy                           0.10       750
   macro avg       0.01      0.10      0.02       750
weighted avg       0.01      0.10      0.02       750



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<center><h1 style="font-size:300%; font-family:cursive; color:black; background:skyblue; padding:15px; border:solid;">REFERENCES & CREDITS</h1></center>

<ol>
    <li style="font-size:150%;"><a href="https://www.reddit.com/r/MachineLearning/comments/ao23cp/p_how_to_use_bert_in_kaggle_competitions_a/">How to use BERT in Kaggle competitions - Reddit Thread</a></li>
    <li style="font-size:150%;"><a href="http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/">A visual guide to using BERT by Jay Alammar</a></li>
    <li style="font-size:150%;"><a href="https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/">Demystifying BERT: Groundbreaking NLP Framework by Mohd Sanad Zaki Rizvi</a></li>
    <li style="font-size:150%;"><a href="https://towardsdatascience.com/bert-for-dummies-step-by-step-tutorial-fb90890ffe03">BERT for Dummies step by step tutorial by Michel Kana</a></li>
</ol>

<br>
<center><h1 style="font-size:300%; font-family:cursive; color:black; background:skyblue; padding:15px; border:solid;">CONCLUSION</h1></center>

<p style="font-size:150%; font-family:verdana;">BERT is undoubtedly a breakthrough in the use of Machine Learning for Natural Language Processing. The fact that it’s approachable and allows fast fine-tuning will likely allow a wide range of practical applications in the future. In this Notebook we have discussed about BERT (Theoritical + Practical Part).</p>

<center><h1 style="font-size:200%; color:green;">Please give this kernel an UPVOTE to show your appreciation, if you find it useful.</h1></center>